In [1]:
from utils.utils import (get_environment,get_root_path,update_storage_access)
import os
import s3fs
from classes.data.satellite_image import SatelliteImage
import numpy as np

In [11]:
update_storage_access()
root_path = get_root_path()
environment = get_environment()

bucket = environment["bucket"]
path_s3 = environment["sources"]["SENTINEL2"][2022]["976"]
path_local = os.path.join(
    root_path, environment["local-path"]["SENTINEL2"][2022]["976"]
)

fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"}
)
fs.download(
    rpath=f"{bucket}/{path_s3}", lpath=f"{path_local}", recursive=True
)

In [ ]:
# path_local = 'SENTINEL2/MAYOTTE/TUILES_2021'
list_names = os.listdir(path_local)
list_paths = [path_local + '/' + name for name in list_names]

nb_images = len(list_paths)
nb_bands = 12
mean = [0]*nb_bands
std = [0]*nb_bands

for path in list_paths:
    si = SatelliteImage.from_raster(
        file_path=path,
        dep=976,
        date=2021,
        n_bands=nb_bands
    )

    for i in range(nb_bands):
        if not np.isnan(np.mean(si.array[i])):
            mean[i] += np.mean(si.array[i])
            std[i] += np.std(si.array[i])

for i in range(nb_bands):
    mean[i] /= nb_images
    std[i] /= nb_images

mean=tuple(mean)
std=tuple(std)

print(mean, std)